In [1]:
import cv2

In [2]:
def get_contours(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7,7), 20)
    # _, thresh = cv2.threshold(gray, 127, 255, 0)

    edged = cv2.Canny(blur,40,100,L2gradient=True)
    dilated = cv2.dilate(edged, (7,7), iterations=4)

    # cv::Mat structuringElement = cv::getStructuringElement(cv::MORPH_ELLIPSE, cv::Size(40, 40));
    # cv::morphologyEx( inputImage, outputImage, cv::MORPH_CLOSE, structuringElement );

    structuringElement = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    dilated = cv2.morphologyEx(edged, cv2.MORPH_DILATE, structuringElement)

    contours, _ = cv2.findContours(dilated,cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    cv2.imshow('dilated', dilated)
    cv2.imshow('canny', edged)
    
    
    print('NUMERO DE CONTORNOS = ', len(contours))

    return contours

In [3]:
def cross_detection(image):

    contours = get_contours(image)

    img = image.copy()
    cnt_img = image.copy()
    cv2.drawContours(cnt_img, contours, -1, (0, 255, 0), 3)
    cv2.imshow('cnt', cnt_img)
    cv2.waitKey(0)


    for i, cnt in enumerate(contours):

        print("CONTOUR ", i)
        
        # Aproxima a um poligono

        perimeter = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.01*perimeter, True)



        hull = cv2.convexHull(cnt, returnPoints=False)
        defects = cv2.convexityDefects(cnt, hull)

        if defects is None:
            continue

        print("DEF = ", defects.shape[0], '\t POL = ', len(approx), '\t', cv2.isContourConvex(cnt))
        
        
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(cnt[s][0])
            end = tuple(cnt[e][0])
            far = tuple(cnt[f][0])
            cv2.line(img,start,end,[0,255,0],2)
            cv2.circle(img,far,5,[0,0,255],-1)

        
        if (8 <= len(approx) <= 14) and (cv2.isContourConvex(cnt) == False):
            print('CRUZ DETECTADA')

        print('\n')

        # checar convex
        # checar numero de vertices
    return img


In [4]:
image = cv2.imread('image/landing_pad_1.png')
cv2.imshow('initial_image', image)
img = cross_detection(image)
cv2.imshow('final_image', img)
cv2.waitKey(0)


NUMERO DE CONTORNOS =  9
CONTOUR  0
DEF =  7 	 POL =  2 	 False


CONTOUR  1
DEF =  15 	 POL =  12 	 False
CRUZ DETECTADA


CONTOUR  2
DEF =  20 	 POL =  12 	 False
CRUZ DETECTADA


CONTOUR  3
DEF =  26 	 POL =  9 	 False
CRUZ DETECTADA


CONTOUR  4
DEF =  7 	 POL =  3 	 False


CONTOUR  5
DEF =  81 	 POL =  16 	 False


CONTOUR  6
DEF =  83 	 POL =  16 	 False


CONTOUR  7
DEF =  21 	 POL =  4 	 False


CONTOUR  8
DEF =  20 	 POL =  4 	 False


